In [0]:
train_b= spark.table("creditrisk_catalog.bronze_creditrisk.app_train")
test_b = spark.table("creditrisk_catalog.bronze_creditrisk.app_test")

In [0]:
from pyspark.sql import functions as F

# Remove duplicates
train_silver = train_b.dropDuplicates(["SK_ID_CURR"])
test_silver = test_b.dropDuplicates(["SK_ID_CURR"])

# Write to Silver tables
train_silver.write.format('delta').mode('overwrite').saveAsTable('creditrisk_catalog.silver_creditrisk.app_train')
test_silver.write.format('delta').mode('overwrite').saveAsTable('creditrisk_catalog.silver_creditrisk.app_test')

print("Silver tables created: creditrisk_catalog.silver_creditrisk.app_train and workspace.silver_creditrisk.app_test")

In [0]:
source_tables = [
    "bureau","bureau_balance",
    "credit_card_balance","installments_payments",
    "POS_CASH_balance","previous_application"
    ]

for t in source_tables:
    df = spark.table(f"workspace.bronze_creditrisk.{t}")
    (df.write.format('delta')
    .mode('overwrite')
    .saveAsTable(f"creditrisk_catalog.silver_creditrisk.{t}"))

In [0]:
%sql
OPTIMIZE creditrisk_catalog.silver_creditrisk.app_train
ZORDER BY (SK_ID_CURR);

OPTIMIZE creditrisk_catalog.silver_creditrisk.app_test
ZORDER BY (SK_ID_CURR);

OPTIMIZE creditrisk_catalog.silver_creditrisk.bureau
ZORDER BY (SK_ID_CURR, SK_ID_BUREAU);

OPTIMIZE creditrisk_catalog.silver_creditrisk.bureau_balance
ZORDER BY (SK_ID_BUREAU);

OPTIMIZE creditrisk_catalog.silver_creditrisk.pos_cash_balance
ZORDER BY (SK_ID_CURR);

OPTIMIZE creditrisk_catalog.silver_creditrisk.credit_card_balance
ZORDER BY (SK_ID_CURR);

OPTIMIZE creditrisk_catalog.silver_creditrisk.installments_payments
ZORDER BY (SK_ID_CURR);


In [0]:
%sql 

VACUUM creditrisk_catalog.silver_creditrisk.app_train RETAIN 168 HOURS;
VACUUM creditrisk_catalog.silver_creditrisk.app_test RETAIN 168 HOURS;
VACUUM creditrisk_catalog.silver_creditrisk.bureau RETAIN 168 HOURS;
VACUUM creditrisk_catalog.silver_creditrisk.bureau_balance RETAIN 168 HOURS;
VACUUM creditrisk_catalog.silver_creditrisk.pos_cash_balance RETAIN 168 HOURS;
VACUUM creditrisk_catalog.silver_creditrisk.credit_card_balance RETAIN 168 HOURS;
VACUUM creditrisk_catalog.silver_creditrisk.installments_payments RETAIN 168 HOURS;
VACUUM creditrisk_catalog.silver_creditrisk.previous_application RETAIN 168 HOURS;